In [1]:
import openai
from openai import AzureOpenAI, AsyncAzureOpenAI
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import requests
import base64

GPT_3_5_TURBO = "gpt-3.5-turbo"
GPT_4_TURBO_PREVIEW = "gpt-4-turbo-preview"
GPT_4 = 'gpt-4'
GPT_4o = 'gpt-4o'

def get_openai_api_key():
    _ = load_dotenv(find_dotenv())

    return os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = get_openai_api_key()
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [2]:
import json


EVAL_FOLDER = '../../data/russian-english/cards/eval_results/ru_eng_'
def get_eval_results_from_file(file_name):
    data = []
    file_path = EVAL_FOLDER + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [3]:
file_names = [f'sm1_new_kap{i}.json' for i in range(1, 10)]

In [4]:
eval_res_1 = get_eval_results_from_file(file_names[1])

In [5]:
import json


OUTPUT_FOLDER = '../../data/russian-english/cards/test_cards/ru_eng_'
def get_cards_from_file(file_name):
    data = []
    file_path = OUTPUT_FOLDER + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [6]:
i = 2
eval_res_1 = get_eval_results_from_file(file_names[i])
cards_1 = get_cards_from_file(file_names[i])

In [7]:
sum([res['translationAccuracy']['isCorrect'] for res in eval_res_1 if res['translationAccuracy']['isCorrect'] is not None]) / len([res['translationAccuracy']['isCorrect'] for res in eval_res_1])

KeyError: 'translationAccuracy'

In [8]:
import json


FOLDER_WITH_JSON = '../../data/russian-finnish/cards/curated_platform_cards/'
def load_cards_from_file(file_name):
    data = []
    file_path = FOLDER_WITH_JSON + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [9]:
all_ru_finn_cards = []
for file_name in tqdm(file_names):   
    ru_finn_cards = load_cards_from_file(file_name)
    for card in ru_finn_cards:
        if card['sentenceFirstLang'] == '':
            all_ru_finn_cards.append(card)

  0%|          | 0/9 [00:00<?, ?it/s]

In [15]:
file_names = [f'sm1_new_kap{i}.json' for i in range(1, 8)]

kek = []
all_cards = []
for file_name in tqdm(file_names):
    ru_cards = get_cards_from_file(file_name)
    eval_results = get_eval_results_from_file(file_name)
    
    lol = []
    for a, b in list(zip(ru_cards, eval_results)):
        if a['sentenceFirstLang'] != '':
            try:
                b['translationAccuracy'] = b['TranslationAccuracy']
            except Exception:
                pass
            lol.append(b)
            
    print(sum([res['translationAccuracy']['isCorrect'] for res in lol]) / len([res['translationAccuracy']['isCorrect'] for res in lol]))

  0%|          | 0/7 [00:00<?, ?it/s]

0.96875
0.9908256880733946
0.9530201342281879
0.9842105263157894
0.9230769230769231
0.9560439560439561
0.91875


In [73]:
kek

[{'RussianSentence': {'isCorrect': True,
   'explanation': "The Russian sentence 'Я дома.' is grammatically correct and natural.",
   'suggestedFix': None},
  'EnglishSentence': {'isCorrect': True,
   'explanation': "The English sentence 'I will be at home.' is grammatically correct and natural.",
   'suggestedFix': None},
  'translationAccuracy': {'isCorrect': False,
   'explanation': "The translation does not match as the Russian sentence means 'I am at home' and the English sentence means 'I will be at home.'",
   'suggestedFix': 'I am at home.'},
  'RussianWordUsage': {'isCorrect': False,
   'explanation': "The word 'быть' does not appear in any form in the Russian sentence.",
   'suggestedFixSentence': 'Я буду дома.',
   'suggestedFixWord': None},
  'EnglishWordUsage': {'isCorrect': True,
   'explanation': "The word 'be' appears in the English sentence in the form 'will be'.",
   'suggestedFix': None},
  'id': 1006},
 {'RussianSentence': {'isCorrect': True,
   'explanation': "The 

In [14]:
id_to_card = {card["id"]: card for card in all_cards}

In [67]:
id_to_card[1043]

{'wordFirstLang': 'вторник',
 'sentenceFirstLang': '',
 'wordSecondLang': 'Tuesday',
 'sentenceSecondLang': '',
 'id': 1043}

In [19]:
id_to_ru_finn_card = {card["id"]: card for card in all_ru_finn_cards}

In [65]:
id_to_ru_finn_card[1031]

{'wordFirstLang': 'Пожалуйста!',
 'sentenceFirstLang': '',
 'wordSecondLang': 'Ole hyvä!',
 'sentenceSecondLang': '',
 'id': 1031}

In [37]:
new_user_prompt = f"""
You will be given a simple word or phrase in Russian, along with its Finnish translation. Your task is to translate this word or phrase into English. Do not translate it literally; instead, ensure that the translation preserves the context and meaning from both the Russian and Finnish versions, and sounds natural in English.

===
Given word or phrase in Russian: '{russian_word}'
===
Finnish translation: '{finnish_translation}'
===

Return only the English translation of the word or phrase and nothing else.
"""

NameError: name 'russian_word' is not defined

In [56]:
def get_card(russian_word, finnish_translation, model=GPT_4o):
    system_prompt = '''You are a multilingual assistant who is proficient in Russian, Finnish and English.'''
    
    user_prompt = f"""
    You will be given a simple word or phrase in Russian, along with its Finnish translation. Your task is to translate this word or phrase into English. Do not translate it literally; instead, ensure that the English translation captures the context and meaning from both the Russian and Finnish versions, and sounds natural in English.

    ---

    Instructions:

    - Consider Both Versions: Carefully read the Russian word or phrase and its Finnish translation to understand the full context and nuances.
    - Meaning Over Literal Translation: Focus on conveying the intended meaning rather than providing a word-for-word translation.
    - Natural Language: Ensure that the English translation sounds natural and fluent, as if spoken or written by a native English speaker.
    - Preserve Context: Reflect any idiomatic expressions or cultural references present in the original phrases.
    - Conciseness: Keep the translation concise and to the point.

    ---

    Input:

    ===
    Given word or phrase in Russian: '{russian_word}'
    ===
    Finnish translation: '{finnish_translation}'
    ===

    ---
    
    Output:

    Return only the English translation of the word or phrase and nothing else.
    """
    
    response = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
      ]
    )

    tr_word = response.choices[0].message.content.strip()
    return tr_word

In [61]:
get_card('15 градусов. холодно.', 'On -15 astetta. On kylmä.').strip('"')

"It's -15 degrees. It's cold."

In [62]:
get_eval('15 градусов. холодно.', 'On -15 astetta. On kylmä.', "It's -15 degrees. It's cold.")

{'translationAccuracy': {'isCorrect': True,
  'explanation': "The English translation 'It's -15 degrees. It's cold.' is accurate and correctly reflects the meanings of both the Russian and Finnish phrases. The context and nuances of the temperature being -15 degrees and it being cold are preserved in the English translation.",
  'suggestedFix': None}}

In [47]:
prompt = """
You are given a word or phrase in Russian, its correct translation in Finnish, and its translation in English. Your task is to evaluate the correctness of the English translation.

Your Responsibilities:

1. Evaluate Translation Accuracy:
   - Verify whether the English translation accurately reflects the meaning of the Russian word or phrase.
   - Verify whether the English translation accurately reflects the meaning of the Finnish translation.
   - Ensure that the English translation preserves the context and nuances from both the Russian and Finnish versions.

2. Assess Naturalness:
   - Check if the English translated word or phrase sounds natural.
   - Confirm that it maintains the intended meaning from the Russian and Finnish versions.

3. Provide Suggestions if Necessary:
   - If the translation isn't accurate or doesn't sound natural, suggest a fix.
   - Explain why you consider it a mistake.

Instructions for Suggested Fixes:

- In the suggestedFix field, provide only the final corrected English translation without any explanations or instructions.
- If there's no need to fix the translation, set suggestedFix to null.

Provide a detailed evaluation for each point and suggest fixes where necessary.

Here are the words or phrases:

------
Word or Phrase in Russian: {card['wordFirstLang']}
------

------
Word or Phrase in Finnish: {card['wordSecondLang']}
------

------
Word or Phrase in English: {card['wordThirdLang']}
------

Respond in JSON format with the following structure:

{{
  "translationAccuracy": {{
    "isCorrect": true/false,
    "explanation": "Detailed explanation if there is an issue or why it's correct",
    "suggestedFix": "Suggested correction if there is an issue or null if no correction is needed"
  }}
}}
"""

In [48]:
def get_eval(ru, finn, eng, model=GPT_4o):
    system_prompt = '''You are a multilingual assistant who is proficient in Russian, Finnish and English.'''
    user_prompt = f"""
    You are given a word or phrase in Russian, its correct translation in Finnish, and its translation in English. Your task is to evaluate the correctness of the English translation.

    Your Responsibilities:

    1. Evaluate Translation Accuracy:
       - Verify whether the English translation accurately reflects the meaning of the Russian word or phrase.
       - Verify whether the English translation accurately reflects the meaning of the Finnish translation.
       - Ensure that the English translation preserves the context and nuances from both the Russian and Finnish versions.

    2. Assess Naturalness:
       - Check if the English translated word or phrase sounds natural.
       - Confirm that it maintains the intended meaning from the Russian and Finnish versions.

    3. Provide Suggestions if Necessary:
       - If the translation isn't accurate or doesn't sound natural, suggest a fix.
       - Explain why you consider it a mistake.

    Instructions for Suggested Fixes:

    - In the suggestedFix field, provide only the final corrected English translation without any explanations or instructions.
    - If there's no need to fix the translation, set suggestedFix to null.

    Provide a detailed evaluation for each point and suggest fixes where necessary.

    Here are the words or phrases:

    ------
    Word or Phrase in Russian: {ru}
    ------

    ------
    Word or Phrase in Finnish: {finn}
    ------

    ------
    Word or Phrase in English: {eng}
    ------

    Respond in JSON format with the following structure:

    {{
      "translationAccuracy": {{
        "isCorrect": true/false,
        "explanation": "Detailed explanation if there is an issue or why it's correct",
        "suggestedFix": "Suggested correction if there is an issue or null if no correction is needed"
      }}
    }}
    """
    
    response = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
      ]
    )
    
    return json.loads(response.choices[0].message.content.strip())

In [22]:
import numpy as np

indices = np.random.choice(np.arange(1000, 2700), size=30)

In [23]:
file_names = [f'sm1_new_kap{i}.json' for i in range(1, 9)]

for file_name in tqdm(file_names):
    ru_cards = get_cards_from_file(file_name)
    eval_results = get_eval_results_from_file(file_name)
    
    for ru_card, eval_res in list(zip(ru_cards, eval_results)):
        if ru_card["id"] in indices:
            print(ru_card)
            print('------')
            print(eval_res)
            print('=======================================')
            print('=======================================')
            print('=======================================')
        

  0%|          | 0/8 [00:00<?, ?it/s]

{'wordFirstLang': 'Как твоё имя?', 'sentenceFirstLang': 'Привет, как твоё имя?', 'wordSecondLang': "What's your name?", 'sentenceSecondLang': "Hello, what's your name?", 'id': 1014}
------
{'RussianSentence': {'isCorrect': True, 'explanation': "The sentence 'Привет, как твоё имя?' is grammatically correct and natural.", 'suggestedFix': None}, 'EnglishSentence': {'isCorrect': True, 'explanation': "The sentence 'Hello, what's your name?' is grammatically correct and natural.", 'suggestedFix': None}, 'translationAccuracy': {'isCorrect': True, 'explanation': "The translation 'Привет, как твоё имя?' for 'Hello, what's your name?' is accurate and natural.", 'suggestedFix': None}, 'RussianWordUsage': {'isCorrect': True, 'explanation': "The word 'Как твоё имя?' appears in some form in the sentence 'Привет, как твоё имя?'.", 'suggestedFixSentence': None, 'suggestedFixWord': None}, 'EnglishWordUsage': {'isCorrect': True, 'explanation': "The word 'What's your name?' appears in some form in the se

In [26]:
file_names = [f'sm1_new_kap{i}.json' for i in range(1, 9)]

for file_name in tqdm(file_names):
    ru_cards = get_cards_from_file(file_name)
    eval_results = get_eval_results_from_file(file_name)
    
    lol = []
    for a, b in list(zip(ru_cards, eval_results)):
        lol.append(b)
            
    print(sum([res['translationAccuracy']['isCorrect'] for res in lol]) / len(lol))

  0%|          | 0/8 [00:00<?, ?it/s]

0.9666666666666667
0.9739583333333334
0.9117647058823529
0.9363295880149812
0.8947368421052632
0.9497907949790795
0.8913043478260869
0.9034749034749034


In [39]:
[res for res in eval_res_1 if not res['translationAccuracy']['isCorrect']]

[{'RussianSentence': {'isCorrect': True,
   'explanation': "The Russian sentence 'Я дома.' is grammatically correct and natural.",
   'suggestedFix': None},
  'EnglishSentence': {'isCorrect': True,
   'explanation': "The English sentence 'I will be at home.' is grammatically correct and natural.",
   'suggestedFix': None},
  'translationAccuracy': {'isCorrect': False,
   'explanation': "The translation does not match as the Russian sentence means 'I am at home' and the English sentence means 'I will be at home.'",
   'suggestedFix': 'I am at home.'},
  'RussianWordUsage': {'isCorrect': False,
   'explanation': "The word 'быть' does not appear in any form in the Russian sentence.",
   'suggestedFixSentence': 'Я буду дома.',
   'suggestedFixWord': None},
  'EnglishWordUsage': {'isCorrect': True,
   'explanation': "The word 'be' appears in the English sentence in the form 'will be'.",
   'suggestedFix': None},
  'id': 1006},
 {'RussianSentence': {'isCorrect': True,
   'explanation': "The 

In [61]:
def correcting_cards_with_eval_results(cards, results, first_lang='Russian', second_lang='Finnish', model=GPT_4o):
    final_cards = []
    for card, result in tqdm(list(zip(cards, results))):
        if result['translationAccuracy']['isCorrect']:
            final_cards.append(card)
            continue
        
        try:
            word, sentence, eng_word, eng_sentence, card_id = card.values()
            explanation = result['translationAccuracy']['explanation']
            suggestedFix = result['translationAccuracy']['suggestedFix']

            system_prompt = f"""
            You are a language expert proficient in both {first_lang} and {second_lang}."""
            
            user_prompt = f'''You will get word or phrase and sentence in Russian and bad translation of word or phrase and sentence in English.
            You have to fix the translation of word or phrase and the translation of the sentence with explanation and suggested fix. 
            Explanation and suggested fix maybe also wrong, in this case just generate correct translation of the  word or phrase and translation of the sentence.
            If sentence is only word or phrase it is okay, just translate this word or phrase.

            === 
            Given word or phrase Russian: '{word}'
            ===
            Given sentence Russian: '{sentence}'
            ===
            Given word or phrase English: '{eng_word}'
            ===
            Given sentence English: '{eng_sentence}'
            ===
            Given explanation: '{explanation}'
            ===
            Given suggested fix: '{suggestedFix}'
            ===

            The english translated sentence should contain english translated word.
            Make sure that thanslated sentence is grammatically correct and natural.
            If the word doesn't appear in some form, suggest using synonyms or related terms in the 
            translation to ensure the sentence remains natural and accurate. You can change either the sentence translation 
            or the word translation. For example, if the word is "досуг," it can be translated as "досуг" or "свободное время". So if the word "свободное время" 
            is in some form used in the sentence and it sounds natural, suggested fix can be to change the word translation to "досуг, свободное время", 
            keeping the sentence intact.
            If the sentence and the word the same, it means the word is phrase and return translation of the word as the word and as the sentence


            Return only english translated word and english translated sentence using semicolon as seperator and nothing else.
            '''

            response = client.chat.completions.create(
              model=model,
              messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
              ]
            )

            tr_word, tr_sentence = response.choices[0].message.content.split(';')

            card = {}
            card["wordFirstLang"] = word
            card["sentenceFirstLang"] = sentence
            card["wordSecondLang"] = tr_word.strip()
            card["sentenceSecondLang"] = tr_sentence.strip()
            card["id"] = card_id
            cards.append(card)
        except Exception as e:
            print('Failed: {}'.format(repr(e)))
    return final_cards

In [63]:
final_cards_1 = correcting_cards_with_eval_results(cards_1, eval_res_1)

  0%|          | 0/150 [00:00<?, ?it/s]

In [51]:
cards_1[6]

{'wordFirstLang': 'быть',
 'sentenceFirstLang': 'Я дома.',
 'wordSecondLang': 'be',
 'sentenceSecondLang': 'I will be at home.',
 'id': 1006}

In [ ]:
def improve_cards(basic_cards, basic_eval_results, system_prompt, first_lang='Russian', second_lang='English', model=GPT_4o):
    cards = []
    eval_results = []
    for basic_card, basic_eval_result in tqdm(list(zip(basic_cards, basic_eval_results))):
        try:
            word, sentence, _, _, card_id = basic_card.values()
            if sentence != '':
                cards.append(basic_card)
                eval_results.append(basic_eval_result)
                continue

            user_prompt = f'''You will get simple word or phrase on Russian. Your task is to translate this word or phrase into English.

            === 
            Given word or phrase: '{word}'
            ===
            Given sentence: '{sentence}'
            ===


            Return only english translated word or phrase and nothing else.
            '''

            response = client.chat.completions.create(
              model=model,
              messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
              ]
            )
            
            tr_word = response.choices[0].message.content

            card = {}
            card["wordFirstLang"] = word
            card["sentenceFirstLang"] = ""
            card["wordSecondLang"] = tr_word.strip()
            card["sentenceSecondLang"] = ""
            card["id"] = card_id
            cards.append(card)
            
            user_prompt = f"""
            You are given a word or phrase in {first_lang}, along with its translation in {second_lang}. Your task is to evaluate the correctness of the translation.

            You have to check whether the translation from {second_lang} to {first_lang} is accurate.

            If the translation isn't accurate then suggest fix with explanation why you considered it as a mistake.




            In the suggestedFix fields don't provide explanations or instructions, just provide the final corrected string.
            If it's better to fix word, not sentence, return null for suggestedFixSentence.


            Provide a detailed evaluation for each point and suggest fixes where necessary.

            Here is the words or phrases:
            
            ======
            Word in {first_lang}: {card['wordFirstLang']}
            ======
            
            ======
            Word in {second_lang}: {card['wordSecondLang']}
            ======

            Respond in JSON format with the following structure:

            {{
              "translationAccuracy": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct",
                "suggestedFix": "Suggested correction if there is an issue"
              }}
            }}
            """
            
            response = client.chat.completions.create(
              model="gpt-4o",
              response_format={ "type": "json_object" },
              messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
              ]
            )

            res = json.loads(response.choices[0].message.content.strip())
            res['id'] = card['id']
            eval_results.append(res)
            
        except Exception as e:
            print(e.message)
    return cards, eval_results